In [32]:
import pandas as pd
import numpy as np
import requests
import json
from config import api_key
from pprint import pprint
from config import gkey

In [33]:
cities_df = pd.read_csv("./redlobster_locations.csv")
cities_df

,CITY,STATE
0,AUBURN,ALABAMA
1,BIRMINGHAM,ALABAMA
2,DECATUR,ALABAMA
3,DOTHAN,ALABAMA
4,FLORENCE,ALABAMA
...,...,...
613,MADISON,WISCONSIN
614,MOUNT PLEASANT,WISCONSIN
615,WAUWATOSA,WISCONSIN
616,CASPER,WYOMING


In [41]:
## Red Lobster ##
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

def searchRedLobster(city:str, state:str) -> requests.Response:

# geocoordinates

    target_search = f"Red Lobster in {city}, {state}"

# set up a parameters dictionary
    params = {
        "query":target_search,
        "key": gkey    
    }

    response = requests.get(base_url, params=params)

    return response

In [42]:
responses = [searchRedLobster(row.CITY, row.STATE) for _, row in cities_df.iterrows()]

In [43]:
test_response = responses[-1].json()['results'][0]

In [44]:
test_response

{'business_status': 'OPERATIONAL',
 'formatted_address': '1923 Dell Range Blvd, Cheyenne, WY 82009, United States',
 'geometry': {'location': {'lat': 41.159724, 'lng': -104.794425},
  'viewport': {'northeast': {'lat': 41.16099537989273,
    'lng': -104.7930749201073},
   'southwest': {'lat': 41.15829572010728, 'lng': -104.7957745798927}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
 'id': 'cf9e21ec3aa5d0591a73a76e54433dfbe0fc28e5',
 'name': 'Red Lobster',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1080,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104632343183583177952">Zachary Reichman</a>'],
   'photo_reference': 'CmRaAAAAifIYi995vj0DW5LRufmHGNmZ-wDkOmftnDzWrWrwiQ3xvX_ppXb9LRD1XMqcK69eygn6w0HMHcUYTPTnWuhpNgIrUi1ymFkZA80kbMSYRHb8frIzT2UFNmnl7DVw7mmYEhDHEq7JBFQSaMnOuWw6b6l9GhQDrkzywfdQ5V5pJPrcwUl_l9LJHQ',
   'width': 1920}],
 'place_id': 'ChIJUbrSFKo7b4cRT8WuT9n-5Co',
 'plus_code': {'compound_code': '5654

In [45]:
def formatresponse(test_response): 
    last_results = {
    'Address':test_response["formatted_address"],'Latitude':test_response['geometry']['location']['lat'], 'Longitude':test_response['geometry']['location']['lng'],'Rating':test_response['rating'],'User Rating Count':test_response['user_ratings_total']}
    
    return last_results

In [46]:
flat_responses = []

for response in responses:
    flat_responses.extend(response.json()['results'])

In [47]:
neat_list = [formatresponse(response) for response in flat_responses]

In [48]:
neat_df = pd.DataFrame(neat_list)

In [49]:
neat_df.shape

(836, 5)

In [50]:
unique_restaurants = neat_df.drop_duplicates()

unique_restaurants.head(20)

,Address,Latitude,Longitude,Rating,User Rating Count
0,"1805 Opelika Rd, Auburn, AL 36830, United States",32.626716,-85.443242,4.1,798
1,"3525 Roosevelt Blvd, Birmingham, AL 35235, Uni...",33.605687,-86.635594,4.0,1281
2,"2027 Beltline Rd SW, Decatur, AL 35601, United...",34.577800,-87.019933,4.1,789
3,"3116 Montgomery Hwy, Dothan, AL 36303, United ...",31.252109,-85.427155,4.1,1026
4,"243 Cox Creek Pkwy, Florence, AL 35630, United...",34.834153,-87.630025,4.4,733
5,"1725 Rainbow Dr, Gadsden, AL 35901, United States",33.980628,-86.007564,4.3,1307
6,"1818 University Dr NW, Huntsville, AL 35801, U...",34.741258,-86.599288,4.1,1747
7,"3941 Airport Blvd, Mobile, AL 36608, United St...",30.675613,-88.146068,4.0,1024
8,"300 Eastdale Cir, Montgomery, AL 36117, United...",32.382590,-86.207348,3.8,1154
9,"515 Quintard Drive, Oxford, AL 36203, United S...",33.616783,-85.827221,4.2,828


In [51]:
redlobster_df = unique_restaurants.copy()

redlobster_df.head(20)

,Address,Latitude,Longitude,Rating,User Rating Count
0,"1805 Opelika Rd, Auburn, AL 36830, United States",32.626716,-85.443242,4.1,798
1,"3525 Roosevelt Blvd, Birmingham, AL 35235, Uni...",33.605687,-86.635594,4.0,1281
2,"2027 Beltline Rd SW, Decatur, AL 35601, United...",34.577800,-87.019933,4.1,789
3,"3116 Montgomery Hwy, Dothan, AL 36303, United ...",31.252109,-85.427155,4.1,1026
4,"243 Cox Creek Pkwy, Florence, AL 35630, United...",34.834153,-87.630025,4.4,733
5,"1725 Rainbow Dr, Gadsden, AL 35901, United States",33.980628,-86.007564,4.3,1307
6,"1818 University Dr NW, Huntsville, AL 35801, U...",34.741258,-86.599288,4.1,1747
7,"3941 Airport Blvd, Mobile, AL 36608, United St...",30.675613,-88.146068,4.0,1024
8,"300 Eastdale Cir, Montgomery, AL 36117, United...",32.382590,-86.207348,3.8,1154
9,"515 Quintard Drive, Oxford, AL 36203, United S...",33.616783,-85.827221,4.2,828


In [52]:
redlobster_df.to_csv("./redlobster_data.csv", header=True)